# Regiment

### Introduction:

Special thanks to: http://chrisalbon.com/ for sharing the dataset and materials.

### Step 1. Import the necessary libraries

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local").appName("My Application").getOrCreate()

from pyspark.sql.types import *
from pyspark.sql.functions import *

25/05/08 12:49:31 WARN Utils: Your hostname, neosoft-Latitude-E7270 resolves to a loopback address: 127.0.1.1; using 10.0.62.133 instead (on interface wlp1s0)
25/05/08 12:49:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/08 12:49:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/05/08 12:49:34 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


### Step 2. Create the DataFrame with the following values:

In [ ]:
raw_data = {'regiment': ['Nighthawks', 'Nighthawks', 'Nighthawks', 'Nighthawks', 'Dragoons', 'Dragoons', 'Dragoons', 'Dragoons', 'Scouts', 'Scouts', 'Scouts', 'Scouts'], 
        'company': ['1st', '1st', '2nd', '2nd', '1st', '1st', '2nd', '2nd','1st', '1st', '2nd', '2nd'], 
        'name': ['Miller', 'Jacobson', 'Ali', 'Milner', 'Cooze', 'Jacon', 'Ryaner', 'Sone', 'Sloan', 'Piger', 'Riani', 'Ali'], 
        'preTestScore': [4, 24, 31, 2, 3, 4, 24, 31, 2, 3, 2, 3],
        'postTestScore': [25, 94, 57, 62, 70, 25, 94, 57, 62, 70, 62, 70]}

### Step 3. Assign it to a variable called regiment.
#### Don't forget to name each column

In [2]:
schema = StructType([
    StructField("regiment", StringType(), True),
    StructField("company", StringType(), True),
    StructField("name", StringType(), True),
    StructField("preTestScore", IntegerType(), True),
    StructField("postTestScore", IntegerType(), True)
])

data = [
    ('Nighthawks', '1st', 'Miller', 4, 25),
    ('Nighthawks', '1st', 'Jacobson', 24, 25),
    ('Nighthawks', '2nd', 'Ali', 31, 57),
    ('Nighthawks', '2nd', 'Milner', 2, 62),
    ('Dragoons', '1st', 'Cooze', 3, 70),
    ('Dragoons', '1st', 'Jacon', 4, 25),
    ('Dragoons', '2nd', 'Ryaner', 24, 94),
    ('Dragoons', '2nd', 'Sone', 31, 57),
    ('Scouts', '1st', 'Sloan', 2, 62),
    ('Scouts', '1st', 'Piger', 3, 70),
    ('Scouts', '2nd', 'Riani', 2, 62),
    ('Scouts', '2nd', 'Ali', 3, 70),
]

regiment = spark.createDataFrame(data, schema=schema)

regiment.show(truncate=False)


+----------+-------+--------+------------+-------------+
|regiment  |company|name    |preTestScore|postTestScore|
+----------+-------+--------+------------+-------------+
|Nighthawks|1st    |Miller  |4           |25           |
|Nighthawks|1st    |Jacobson|24          |25           |
|Nighthawks|2nd    |Ali     |31          |57           |
|Nighthawks|2nd    |Milner  |2           |62           |
|Dragoons  |1st    |Cooze   |3           |70           |
|Dragoons  |1st    |Jacon   |4           |25           |
|Dragoons  |2nd    |Ryaner  |24          |94           |
|Dragoons  |2nd    |Sone    |31          |57           |
|Scouts    |1st    |Sloan   |2           |62           |
|Scouts    |1st    |Piger   |3           |70           |
|Scouts    |2nd    |Riani   |2           |62           |
|Scouts    |2nd    |Ali     |3           |70           |
+----------+-------+--------+------------+-------------+



### Step 4. What is the mean preTestScore from the regiment Nighthawks?  

In [5]:
regiment.filter(col("regiment")=="Nighthawks").groupBy("regiment").agg(mean("pretestscore")).show()

+----------+-----------------+
|  regiment|avg(pretestscore)|
+----------+-----------------+
|Nighthawks|            15.25|
+----------+-----------------+



### Step 5. Present general statistics by company

In [7]:
regiment.describe().show()

25/05/08 13:09:10 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+--------+-------+----+------------------+------------------+
|summary|regiment|company|name|      preTestScore|     postTestScore|
+-------+--------+-------+----+------------------+------------------+
|  count|      12|     12|  12|                12|                12|
|   mean|    NULL|   NULL|NULL|11.083333333333334|56.583333333333336|
| stddev|    NULL|   NULL|NULL|12.324833270739004|21.364619577632777|
|    min|Dragoons|    1st| Ali|                 2|                25|
|    max|  Scouts|    2nd|Sone|                31|                94|
+-------+--------+-------+----+------------------+------------------+



### Step 6. What is the mean of each company's preTestScore?

In [8]:
regiment.groupBy("company").agg(mean("pretestscore")).show()

+-------+-----------------+
|company|avg(pretestscore)|
+-------+-----------------+
|    2nd|             15.5|
|    1st|6.666666666666667|
+-------+-----------------+



### Step 7. Present the mean preTestScores grouped by regiment and company

In [9]:
regiment.groupBy("regiment","company").agg(mean("pretestscore")).show()

+----------+-------+-----------------+
|  regiment|company|avg(pretestscore)|
+----------+-------+-----------------+
|Nighthawks|    1st|             14.0|
|  Dragoons|    1st|              3.5|
|Nighthawks|    2nd|             16.5|
|  Dragoons|    2nd|             27.5|
|    Scouts|    2nd|              2.5|
|    Scouts|    1st|              2.5|
+----------+-------+-----------------+



### Step 8. Present the mean preTestScores grouped by regiment and company without heirarchical indexing

In [10]:
regiment.groupBy("regiment","company").agg(mean("pretestscore")).orderBy("regiment").show()

+----------+-------+-----------------+
|  regiment|company|avg(pretestscore)|
+----------+-------+-----------------+
|  Dragoons|    1st|              3.5|
|  Dragoons|    2nd|             27.5|
|Nighthawks|    1st|             14.0|
|Nighthawks|    2nd|             16.5|
|    Scouts|    2nd|              2.5|
|    Scouts|    1st|              2.5|
+----------+-------+-----------------+



### Step 9. Group the entire dataframe by regiment and company

In [16]:
regiment.groupBy("regiment", "company") \
    .agg(avg("preTestScore").alias("average_preTestScore")) \
    .orderBy("regiment")\
    .show()


+----------+-------+--------------------+
|  regiment|company|average_preTestScore|
+----------+-------+--------------------+
|  Dragoons|    1st|                 3.5|
|  Dragoons|    2nd|                27.5|
|Nighthawks|    1st|                14.0|
|Nighthawks|    2nd|                16.5|
|    Scouts|    2nd|                 2.5|
|    Scouts|    1st|                 2.5|
+----------+-------+--------------------+



### Step 10. What is the number of observations in each regiment and company

In [17]:
regiment.agg(count("regiment"), count("company")).show()

+---------------+--------------+
|count(regiment)|count(company)|
+---------------+--------------+
|             12|            12|
+---------------+--------------+



### Step 11. Iterate over a group and print the name and the whole data from the regiment

In [19]:
grouped_data = regiment.groupBy("regiment").agg(*[first(c).alias(c) for c in regiment.columns if c != "regiment"])

for row in grouped_data.collect():
    regiment_name = row['regiment']
    print(f"Regiment: {regiment_name}")
    regiment_data = regiment.filter(regiment.regiment == regiment_name)
    regiment_data.show(truncate=False)


Regiment: Dragoons
+--------+-------+------+------------+-------------+
|regiment|company|name  |preTestScore|postTestScore|
+--------+-------+------+------------+-------------+
|Dragoons|1st    |Cooze |3           |70           |
|Dragoons|1st    |Jacon |4           |25           |
|Dragoons|2nd    |Ryaner|24          |94           |
|Dragoons|2nd    |Sone  |31          |57           |
+--------+-------+------+------------+-------------+

Regiment: Nighthawks
+----------+-------+--------+------------+-------------+
|regiment  |company|name    |preTestScore|postTestScore|
+----------+-------+--------+------------+-------------+
|Nighthawks|1st    |Miller  |4           |25           |
|Nighthawks|1st    |Jacobson|24          |25           |
|Nighthawks|2nd    |Ali     |31          |57           |
|Nighthawks|2nd    |Milner  |2           |62           |
+----------+-------+--------+------------+-------------+

Regiment: Scouts
+--------+-------+-----+------------+-------------+
|regiment